In [1]:
# noexport
!typech survey_asknext.ipynb

[NbConvertApp] Converting notebook survey_asknext.ipynb to python


In [2]:
from train_utils import *
from plot_utils import *
from scipy.stats import chisquare


In [3]:
def get_most_common_key_in_dict(d):
  most_common = None
  for k in sorted(d.keys()):
    if most_common == None:
      most_common = k
    elif d[k] > d[most_common]:
      most_common = k
  return most_common

def get_most_common_survey_result_for_user(user):
  results = get_survey_results_for_user(user)
  return get_most_common_key_in_dict(results)

def get_most_common_survey_result_for_install(install):
  results = get_survey_results_for_install(install)
  return get_most_common_key_in_dict(results)

In [4]:
def get_survey_results_across_all_installs():
  output = Counter()
  for install in get_installs_with_asknext_survey():
    for k,v in get_survey_results_for_install(install).items():
      output[k] += v
  return output

def get_survey_results_across_all_users():
  output = Counter()
  for user in get_users_with_asknext_survey():
    for k,v in get_survey_results_for_user(user).items():
      output[k] += v
  return output

In [5]:
def get_survey_results_for_install(install):
  output = Counter()
  collection_items = get_collection_for_install(install, 'internal:choose_difficulty')
  for item in collection_items:
    if ('developer_mode' in item) and (item['developer_mode'] == True):
      continue
    if ('unofficial_version' in item):
      continue
    if 'action_type' in item and item['action_type'] == 'asknext_chosen':
      asknext = item['asknext']
      output[asknext] += 1
  return output

def get_survey_results_for_user(user):
  output = Counter()
  collection_items = get_collection_for_user(user, 'internal:choose_difficulty')
  for item in collection_items:
    if ('developer_mode' in item) and (item['developer_mode'] == True):
      continue
    if ('unofficial_version' in item):
      continue
    if 'action_type' in item and item['action_type'] == 'asknext_chosen':
      asknext = item['asknext']
      output[asknext] += 1
  return output

In [6]:
def get_installs_with_asknext_survey():
  output = []
  installs_with_difficulty = get_installs_with_choose_difficulty()
  for install in installs_with_difficulty:
    abtest_settings = get_abtest_settings_for_install(install)
    if abtest_settings.get('frequency_of_choose_difficulty') == 'survey':
      output.append(install)
  return output

def get_users_with_asknext_survey():
  output = []
  users_with_difficulty = get_users_with_choose_difficulty()
  for user in users_with_difficulty:
    abtest_settings = get_abtest_settings(user)
    if abtest_settings.get('frequency_of_choose_difficulty') == 'survey':
      output.append(user)
  return output

In [7]:
def get_survey_choice_to_num_installs_who_choose_it_most_commonly():
  output = Counter()
  for install in get_installs_with_asknext_survey():
    most_common_result = get_most_common_survey_result_for_install(install)
    if most_common_result == None:
      continue
    output[most_common_result] += 1
  return output

def get_survey_choice_to_num_users_who_choose_it_most_commonly():
  output = Counter()
  for user in get_users_with_asknext_survey():
    most_common_result = get_most_common_survey_result_for_user(user)
    if most_common_result == None:
      continue
    output[most_common_result] += 1
  return output

In [14]:
def get_most_common_difficulty_for_user(user):
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  max_count = 0
  max_difficulty = None
  for difficulty,count in difficulty_counts.items():
    if count > max_count:
      max_count = count
      max_difficulty = difficulty
  return max_difficulty

def get_survey_choice_to_difficulty_choice_counts():
  output = {
    'nextvisit': Counter(),
    'hour': Counter(),
    'day': Counter(),
    'week': Counter(),
  }
  for user in get_users_with_asknext_survey():
    collection_items = get_collection_for_user(user, 'internal:choose_difficulty')
    for item in collection_items:
      if ('developer_mode' in item) and (item['developer_mode'] == True):
        continue
      if ('unofficial_version' in item):
        continue
      if 'action_type' in item and item['action_type'] == 'asknext_chosen':
        asknext = item['asknext']
        difficulty = item['difficulty']
        output[asknext][difficulty] += 1
        #output[asknext] += 1
  return output

def plot_survey_choice_to_difficulty_choice_counts():
  plot_dictdict_as_bar(
    get_survey_choice_to_difficulty_choice_counts(),
    ylabel = 'Number of times chosen',
    xlabel = 'Intervention difficulty chosen, along with when to ask about difficulty again',
    title = 'Choices for intervention difficulty and when to ask about difficulty again',
    remap_labels = {'easy': 'Easy', 'medium': 'Medium', 'hard': 'Hard', 'nothing': 'Nothing'},
    font=dict(size=18),
  )

def plot_survey_choice_counts_raw_per_user():
  survey_results = get_survey_results_across_all_users()
  print(survey_results)
  print(chisquare(list(survey_results.values())))
  plot_dict_as_bar(
    survey_results,
    ylabel = 'Number of times chosen',
    xlabel = 'Choice for when to ask next about intervention difficulty',
    title = 'Choice for when to ask next about intervention difficulty, raw counts'
  )

def plot_survey_choice_counts_raw_per_install():
  survey_results = get_survey_results_across_all_installs()
  print(survey_results)
  print(chisquare(list(survey_results.values())))
  plot_dict_as_bar(
    survey_results,
    ylabel = 'Number of times chosen',
    xlabel = 'Choice for when to ask next about intervention difficulty',
    title = 'Choice for when to ask next about intervention difficulty, raw counts'
  )

def plot_survey_choice_counts_install_normalized():
  survey_choice_to_num_installs_who_choose_it_most_commonly = get_survey_choice_to_num_installs_who_choose_it_most_commonly()
  print(survey_choice_to_num_installs_who_choose_it_most_commonly)
  print(chisquare(list(survey_choice_to_num_installs_who_choose_it_most_commonly.values())))
  plot_dict_as_bar(
    survey_choice_to_num_installs_who_choose_it_most_commonly,
    ylabel = 'Number of users',
    xlabel = 'User\'s most frequent choice for when to next ask about intervention difficulty',
    title = 'Most frequent choice for when to ask next about intervention difficulty, by number of users',
  )

def plot_survey_choice_counts_user_normalized():
  survey_choice_to_num_users_who_choose_it_most_commonly = get_survey_choice_to_num_users_who_choose_it_most_commonly()
  print(survey_choice_to_num_users_who_choose_it_most_commonly)
  print(chisquare(list(survey_choice_to_num_users_who_choose_it_most_commonly.values())))
  plot_dict_as_bar(
    survey_choice_to_num_users_who_choose_it_most_commonly,
    ylabel = 'Number of users',
    xlabel = 'User\'s most frequent choice for when to next ask about intervention difficulty',
    title = 'Most frequent choice for when to ask next about intervention difficulty, by number of users',
  )

In [15]:
# noexport

plot_survey_choice_to_difficulty_choice_counts()

In [10]:
# noexport

plot_survey_choice_counts_raw_per_install()

Counter({'nextvisit': 19973, 'hour': 2430, 'week': 1815, 'day': 1180})
Power_divergenceResult(statistic=39097.19710213403, pvalue=0.0)


In [11]:
# noexport

plot_survey_choice_counts_raw_per_user()

Counter({'nextvisit': 15873, 'hour': 1915, 'week': 1586, 'day': 1066})
Power_divergenceResult(statistic=30297.985518591002, pvalue=0.0)


In [12]:
# noexport

plot_survey_choice_counts_user_normalized()

Counter({'nextvisit': 210, 'week': 160, 'day': 60, 'hour': 55})
Power_divergenceResult(statistic=144.48453608247422, pvalue=4.077627919509116e-31)


In [13]:
# noexport

plot_survey_choice_counts_install_normalized()

Counter({'nextvisit': 280, 'week': 197, 'day': 77, 'hour': 73})
Power_divergenceResult(statistic=192.56618819776713, pvalue=1.7031449070858785e-41)
